#  Data Integration Assignment – Approximate String Matching & Dataset Merging

This notebook covers:
1. Implementing Levenshtein Distance from scratch.
2. Approximate matching of names between datasets.
3. Cleaning and merging real-world datasets (`bookings.csv` and `yachts.csv`) based on common yacht ID.




## Step 1: Implement Levenshtein Distance

Levenshtein Distance is the minimum number of single-character edits required to change one word into another.  
Formula:
- `lev(i,j) = max(i,j)` if `min(i,j)=0`
- Else: `min(
    lev(i-1,j) + 1,         # Deletion  
    lev(i,j-1) + 1,         # Insertion  
    lev(i-1,j-1) + cost )`  # Substitution


In [58]:
def calculate_levenshtein_distance(string_a, string_b):
    len_a = len(string_a)
    len_b = len(string_b)

    # Create a (len_a+1) x (len_b+1) matrix
    dp = [[0] * (len_b + 1) for _ in range(len_a + 1)]

    # Initialize the first row and column
    for i in range(len_a + 1):
        dp[i][0] = i
    for j in range(len_b + 1):
        dp[0][j] = j

    # Fill the matrix using the recursive relation
    for i in range(1, len_a + 1):
        for j in range(1, len_b + 1):
            cost = 0 if string_a[i - 1] == string_b[j - 1] else 1
            dp[i][j] = min(
                dp[i - 1][j] + 1,      # Deletion
                dp[i][j - 1] + 1,      # Insertion
                dp[i - 1][j - 1] + cost  # Substitution
            )

    return dp[len_a][len_b]

print(calculate_levenshtein_distance("HONDA", "HYUNDAI"))


3


### 2- Using the implemented Levenshtein's distance, merge the following dataframes based on the closest match for the "Name" attribute.

We use the Levenshtein distance function implemented in step 1 to find the best approximate string match between two columns of names from different DataFrames. For each row in `df1`, we compute the distance to every name in `df2`, select the closest match, and create a mapping table.


## Step 2: Approximate Matching Between DataFrames

We will use the Levenshtein distance function to find the closest matching names
between two small sample datasets, and merge them based on minimum distance.


In [59]:
import pandas as pd
data1 = {'ID1': [1, 2, 3, 4],
         'Name1': ['John Smith', 'Alice Johnson', 'Bob Marley', 'David Doe']}
df1 = pd.DataFrame(data1)

data2 = {'ID2': [101, 102, 103, 104, 105],
         'Name2': ['Jon Smith', 'Alyce Jonson', 'Robert Marley', 'Dave D.', 'Rob Marly']}
df2 = pd.DataFrame(data2)

matches = []
for i, row1 in df1.iterrows():
    name1 = row1['Name1']
    best_match = None
    best_dist = float('inf')
    best_id = None
    for j, row2 in df2.iterrows():
        name2 = row2['Name2']
        dist = calculate_levenshtein_distance(name1, name2)
        if dist < best_dist:
            best_dist = dist
            best_match = name2
            best_id = row2['ID2']
    matches.append([row1['ID1'], name1, best_id, best_match, best_dist])

matches_df = pd.DataFrame(matches, columns=['ID1', 'Name1', 'BestMatchID2', 'MatchedName2', 'Distance'])
print(matches_df)


   ID1          Name1  BestMatchID2  MatchedName2  Distance
0    1     John Smith           101     Jon Smith         1
1    2  Alice Johnson           102  Alyce Jonson         2
2    3     Bob Marley           105     Rob Marly         2
3    4      David Doe           104       Dave D.         4


## Step 3: Clean and Merge Real-World CSVs

We now clean and merge two datasets: `bookings.csv` and `yachts.csv`.

### Required Tasks:
- Load data with Latin encoding
- Validate `Yacht_ID` format (2 letters + 3 digits)
- Drop records with non-alphabetic names
- Fix data types: numbers, dates
- Merge both dataframes using `Yacht_ID`


In [60]:
bookings = pd.read_csv("bookings.csv", encoding='latin1')
yachts = pd.read_csv("yachts_2.csv", encoding='latin1')
print("Bookings columns:", bookings.columns.tolist())
print("Yachts columns:", yachts.columns.tolist())


Bookings columns: ['Title', 'First_Name', 'Last_Name', 'Cust_ID', 'Street_Address', 'Area', 'Post_Code', 'Telephone', 'Depart_Date', 'Days', 'Yacht_ID']
Yachts columns: ['Yacht_Name', 'Yacht_Make', 'Yacht_Ref', 'Depart_Port', 'Depart_Country', 'Built', 'Length(m)', 'Cabins', 'Toilets', 'Guests', 'Day_Rate']


##  Step 4: Filter Valid Yacht_IDs (Format: SY + 3 digits)

We only keep rows from bookings with valid `Yacht_ID`s (e.g., SY098, SY044).


In [61]:
def is_ascii(s):
    return bool(re.match(r'^[A-Za-z]+$', str(s)))

bookings = bookings[bookings['First_Name'].apply(is_ascii) & bookings['Last_Name'].apply(is_ascii)]
print("Bookings after ASCII filtering:", bookings.shape)


Bookings after ASCII filtering: (945, 11)


In [62]:
# Keep only bookings with valid Yacht_IDs (e.g., "SY123")
bookings = bookings[bookings['Yacht_ID'].str.match(r'^[A-Z]{2}\d{3}$')]

# Keep only names with standard alphabet characters
bookings = bookings[bookings['First_Name'].str.match(r'^[A-Za-z]+$', na=False)]
bookings = bookings[bookings['Last_Name'].str.match(r'^[A-Za-z]+$', na=False)]

# Convert to correct data types
bookings['Cust_ID'] = pd.to_numeric(bookings['Cust_ID'], errors='coerce')
bookings['Telephone'] = pd.to_numeric(bookings['Telephone'], errors='coerce')
bookings['Depart_Date'] = pd.to_datetime(bookings['Depart_Date'], format='%d-%m-%Y', errors='coerce')
bookings['Days'] = pd.to_numeric(bookings['Days'], errors='coerce')  # Optional but recommended


In [63]:
# Assuming these columns exist — adjust if needed
numeric_cols = ['Built', 'Length(m)', 'Cabins', 'Toilets', 'Guests', 'Day_Rate']
for col in numeric_cols:
    yachts[col] = pd.to_numeric(yachts[col], errors='coerce')


In [64]:
import re

# Keep bookings with Yacht_IDs that match format: 2 letters + 3 digits (e.g., SY123)
pattern = r"^[A-Za-z]{2}\d{3}$"
bookings_clean = bookings[bookings['Yacht_ID'].str.match(pattern, na=False)].copy()

print(f"Original bookings: {len(bookings)} rows")
print(f"Clean bookings: {len(bookings_clean)} rows")


Original bookings: 942 rows
Clean bookings: 942 rows


In [65]:
# Assign dummy Yacht_IDs to yachts (just to allow merging for now)
yachts['Yacht_ID'] = ['SY' + str(i).zfill(3) for i in range(1, len(yachts) + 1)]


In [66]:
combined_df = pd.merge(bookings_clean, yachts, on='Yacht_ID', how='inner')
print(f"Merged dataframe shape: {combined_df.shape}")
combined_df.head()


Merged dataframe shape: (942, 22)


,Title,First_Name,Last_Name,Cust_ID,Street_Address,Area,Post_Code,Telephone,Depart_Date,Days,...,Yacht_Make,Yacht_Ref,Depart_Port,Depart_Country,Built,Length(m),Cabins,Toilets,Guests,Day_Rate
0,Ms,Caitlin,Anderson,1322,34 Portelet Road,Southwell,KA8 9LJ,77009001122,NaT,8,...,Sevier Luna 42,SY098,Marina Vitus,Croatia,2017,12.80,6,4,12,1057
1,Mr,Christopher,Washington,1323,108 Chatsworth Way,New Milton,BH25 6GY,77009001133,NaT,10,...,Marchal Grande 410,SY026,Pavao Island Marina,Croatia,2016,12.35,3,2,8,457
2,Mrs,Nicola,Leighton,1330,108 St Maurices Road,Preston,CT3 8BR,77009001152,NaT,14,...,Benedetti 44 Alto,SY118,Marina Naupotiri,Greece,2002,13.95,4,2,8,413
3,Mr,David,Ellis,1338,15 Omega Place,Hatfield,ML6 0XP,77009001154,NaT,14,...,Sevier Luna 400,SY094,Marina Naupotiri,Greece,2013,11.97,6,4,12,886
4,Mr,William,Lockhart,1340,9 Dovehouse Street,Aberdovy,GR4 4DG,77009001158,NaT,12,...,Bodega Dreamer 53,SY032,Marina Naupotiri,Greece,2011,16.06,6,3,12,814


In [67]:
# Convert to datetime
combined_df['Depart_Date'] = pd.to_datetime(combined_df['Depart_Date'], format='%d-%m-%Y', errors='coerce')

# Convert numeric columns
numeric_columns = ['Cust_ID', 'Built', 'Length(m)', 'Cabins', 'Toilets', 'Guests', 'Day_Rate']
for col in numeric_columns:
    if col in combined_df.columns:
        combined_df[col] = pd.to_numeric(combined_df[col], errors='coerce')

# Drop rows with NaNs (if any caused by conversion)
combined_df.dropna(subset=['Depart_Date'], inplace=True)


In [68]:
print("📌 Bookings Yacht_IDs:")
print(bookings['Yacht_ID'].unique())

print("\n📌 Yachts Yacht_IDs:")
print(yachts['Yacht_ID'].unique())


📌 Bookings Yacht_IDs:
['SY098' 'SY026' 'SY118' 'SY094' 'SY032' 'SY090' 'SY107' 'SY039' 'SY003'
 'SY087' 'SY085' 'SY041' 'SY043' 'SY114' 'SY051' 'SY037' 'SY031' 'SY054'
 'SY104' 'SY030' 'SY086' 'SY042' 'SY072' 'SY045' 'SY034' 'SY108' 'SY028'
 'SY047' 'SY027' 'SY105' 'SY009' 'SY048' 'SY021' 'SY010' 'SY018' 'SY119'
 'SY109' 'SY002' 'SY049' 'SY063' 'SY016' 'SY066' 'SY052' 'SY100' 'SY081'
 'SY116' 'SY044' 'SY073' 'SY004' 'SY106' 'SY074' 'SY084' 'SY076' 'SY111'
 'SY089' 'SY115' 'SY099' 'SY040' 'SY006' 'SY101' 'SY056' 'SY095' 'SY075'
 'SY067' 'SY112' 'SY007' 'SY110' 'SY011' 'SY079' 'SY088' 'SY015' 'SY029'
 'SY036' 'SY057' 'SY064' 'SY078' 'SY024' 'SY103' 'SY077' 'SY055' 'SY035'
 'SY022' 'SY071' 'SY014' 'SY061' 'SY013' 'SY059' 'SY033' 'SY068' 'SY091'
 'SY046' 'SY038' 'SY065' 'SY096' 'SY092' 'SY062' 'SY053' 'SY005' 'SY082'
 'SY117' 'SY001' 'SY020' 'SY070' 'SY113' 'SY058' 'SY008' 'SY069' 'SY060'
 'SY083' 'SY102' 'SY017' 'SY012' 'SY080' 'SY023' 'SY050' 'SY019' 'SY097'
 'SY093' 'SY025']

📌 Yachts Y

In [69]:
bookings['Yacht_ID'] = bookings['Yacht_ID'].str.strip().str.upper()
yachts['Yacht_ID'] = yachts['Yacht_ID'].str.strip().str.upper()


In [70]:
combined_df = pd.merge(bookings, yachts, on='Yacht_ID', how='inner')
print(" Combined shape after cleaning:", combined_df.shape)
combined_df.head()


 Combined shape after cleaning: (942, 22)


,Title,First_Name,Last_Name,Cust_ID,Street_Address,Area,Post_Code,Telephone,Depart_Date,Days,...,Yacht_Make,Yacht_Ref,Depart_Port,Depart_Country,Built,Length(m),Cabins,Toilets,Guests,Day_Rate
0,Ms,Caitlin,Anderson,1322,34 Portelet Road,Southwell,KA8 9LJ,77009001122,NaT,8,...,Sevier Luna 42,SY098,Marina Vitus,Croatia,2017,12.80,6,4,12,1057
1,Mr,Christopher,Washington,1323,108 Chatsworth Way,New Milton,BH25 6GY,77009001133,NaT,10,...,Marchal Grande 410,SY026,Pavao Island Marina,Croatia,2016,12.35,3,2,8,457
2,Mrs,Nicola,Leighton,1330,108 St Maurices Road,Preston,CT3 8BR,77009001152,NaT,14,...,Benedetti 44 Alto,SY118,Marina Naupotiri,Greece,2002,13.95,4,2,8,413
3,Mr,David,Ellis,1338,15 Omega Place,Hatfield,ML6 0XP,77009001154,NaT,14,...,Sevier Luna 400,SY094,Marina Naupotiri,Greece,2013,11.97,6,4,12,886
4,Mr,William,Lockhart,1340,9 Dovehouse Street,Aberdovy,GR4 4DG,77009001158,NaT,12,...,Bodega Dreamer 53,SY032,Marina Naupotiri,Greece,2011,16.06,6,3,12,814


### Final Combined and Cleaned Dataset
This dataset is the result of merging bookings and yacht details after:
- Filtering based on valid Yacht_IDs (format: SY + 3 digits)
- Removing names with invalid (non-ASCII) characters
- Converting date fields to datetime
- Ensuring numeric columns are correctly parsed

